# Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


## Importamos librerias

In [279]:
!pip install numpy scikit-learn

In [280]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

np.random.seed(42) # Setting a seed for reproducibility

## 1 Vectorización de documentos

In [281]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

### Se preparan datos de test y de entrenamiento ya vectorizados

In [282]:
tfidfvect = TfidfVectorizer(
    stop_words="english",  # quitar stopwords en inglés
    strip_accents='ascii',  # elimina acentos y caracteres especiales,
)

# `X_train` la podemos denominar como la matriz documento-término
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target


In [283]:
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

shape: (11314, 101320)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101320


In [284]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

### Selecciono 5 indices random entre 0 y X_train.shape[0]

In [285]:
idx_to_compare = np.random.randint(0, X_train.shape[0], 5)
print(idx_to_compare)

[ 7270   860  5390  5191 11284]


In [286]:
# revisamos los documentos elegidos
for idx in idx_to_compare:
    print(newsgroups_train.data[idx])
    print("------------------------")


I think Murray has done a great job.  He's picked up Ciccarelli,
Sheppard, Ysebaert, Howe, Coffey, and Riendeau (plus some depth players) 
without giving up anything the Wings needed or any of his top prospects.
All of this in three years.  Has anyone done better?

The year before he took over, the Wings didn't even make the playoffs.
There was about a year and a half during Demers' stint that the Wings
did OK, but that was due to Demers' motavational skills and clutch
and grab style.  They didn't have much talent.

Gerald, Murray wasn't responsible for Primeau (although I'm not
ready to admit that's a horrible pick).  They hired him after the
draft (which has never made sense to me).  His first pick was
Lapointe.

Ron 
------------------------



This is an interesting question to ponder.  Did Brad/Ali's sickness
make Ayatollah-style Islam attractive to him or did this new religion 
that Brad/Ali has formally adopted give him this sickness?

------------------------


Specifically, w

### Similaridad de documentos

In [287]:
cossin_by_idx = {}
# midamos la similaridad coseno con todos los documentos de train
for idx in idx_to_compare:
    cossin_by_idx[idx] = {
                          'value':np.sort(cosine_similarity(X_train[idx], X_train)[0])[::-1],
                          'idx':np.argsort(cosine_similarity(X_train[idx], X_train)[0])[::-1]
                          }

In [288]:
for idx in idx_to_compare:
    print(f'Documento {idx} -- Clase {newsgroups_train.target_names[y_train[idx]]}')
    print(f"--Similitud coseno de los 5 mas similares {cossin_by_idx[idx]['value'][1:6]}")
    print(f"--Indices de los 5 mas similares {cossin_by_idx[idx]['idx'][1:6]}")
    print(f"--clases de los 5 mas similares {[newsgroups_train.target_names[y_train[i]]  for i in cossin_by_idx[idx]['idx'][1:6]]}\n")

Documento 7270 -- Clase rec.sport.hockey
--Similitud coseno de los 5 mas similares [0.19859162 0.19737357 0.18785384 0.16566917 0.16552367]
--Indices de los 5 mas similares [11294  9410  9756  3463  2852]
--clases de los 5 mas similares ['rec.sport.hockey', 'rec.sport.hockey', 'rec.sport.hockey', 'rec.sport.hockey', 'rec.sport.hockey']

Documento 860 -- Clase talk.politics.mideast
--Similitud coseno de los 5 mas similares [0.18795448 0.1622525  0.15750032 0.15624314 0.15564764]
--Indices de los 5 mas similares [5825 2551  899 7714 6920]
--clases de los 5 mas similares ['talk.politics.mideast', 'sci.crypt', 'talk.politics.mideast', 'comp.windows.x', 'talk.politics.mideast']

Documento 5390 -- Clase alt.atheism
--Similitud coseno de los 5 mas similares [0.23739186 0.19208811 0.1500076  0.14766618 0.1460887 ]
--Indices de los 5 mas similares [   20 10254  3845  1137  6376]
--clases de los 5 mas similares ['alt.atheism', 'alt.atheism', 'alt.atheism', 'alt.atheism', 'alt.atheism']

Document

### Conclusión

Tras aplicar limpieza de texto y filtros en los datos (eliminación de stopwords, conversión a minúsculas( automatico por TfidfVectorizer) y eliminando acentos/ caracteres especiales), se observa que los documentos tienden a agruparse de manera más coherente con su categoría original. Por ejemplo, los cinco vecinos más similares de los documentos de rec.sport.hockey y alt.atheism pertenecen todos a la misma clase, lo que evidencia una mejor representación temática. Aunque en algunos casos aparecen vecinos de otras categorías, la limpieza del texto ayuda a que la similitud coseno refleje de manera más precisa la relación entre documentos de la misma temática.

## 2 Modelo de clasificación Naïve Bayes

#### MultinomialNB

In [289]:
clf = MultinomialNB()

clf.fit(X_train, y_train)

y_pred =  clf.predict(X_test)

In [290]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.6467991505900852

#### ComplementNB

In [291]:
 clf_complement = ComplementNB()

 clf_complement.fit(X_train, y_train)

 y_pred_complement = clf_complement.predict(X_test)

In [292]:
f1_score(y_test, y_pred_complement, average='macro')

0.6936107849650025

## 3 Matriz término-documento

In [293]:
X_train_T = X_train.T
print(f'Shape transpuesta: {X_train_T.shape}')  # Ahora filas = términos, columnas = documentos

Shape transpuesta: (101320, 11314)


In [294]:
words_to_compare = ["hockey", "game", "computer", "space", "god"]

word_indices = [tfidfvect.vocabulary_[w] for w in words_to_compare]

In [295]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [296]:
cossin_by_word = {}

for idx, word in zip(word_indices, words_to_compare):
    # vector de la palabra
    vec = X_train_T[idx]
    # similitud coseno con todas las palabras
    sim = cosine_similarity(vec, X_train_T)[0]
    # orden descendente
    sorted_idx = np.argsort(sim)[::-1]
    sorted_val = np.sort(sim)[::-1]

    cossin_by_word[word] = {
        "value": sorted_val[1:6],   # omitimos la palabra misma
        "idx": sorted_idx[1:6]
    }

# Mostrar resultados
for word in words_to_compare:
    similar_words = [idx2word[i] for i in cossin_by_word[word]["idx"]]
    print(f"Palabra: {word}")
    print(f"-- Similitud coseno con las 5 más similares: {cossin_by_word[word]['value']}")
    print(f"-- Palabras más similares: {similar_words}\n")

Palabra: hockey
-- Similitud coseno con las 5 más similares: [0.27239818 0.26742755 0.24160446 0.23577144 0.21826898]
-- Palabras más similares: ['ncaa', 'nhl', 'affiliates', 'xenophobes', 'sportschannel']

Palabra: game
-- Similitud coseno con las 5 más similares: [0.21447335 0.19187035 0.18366852 0.18007694 0.17360628]
-- Palabras más similares: ['games', 'espn', 'team', 'hockey', 'scored']

Palabra: computer
-- Similitud coseno con las 5 más similares: [0.15792817 0.1531366  0.1531366  0.14207635 0.13664679]
-- Palabras más similares: ['decwriter', 'deluged', 'harkens', 'shopper', 'delicate']

Palabra: space
-- Similitud coseno con las 5 más similares: [0.32793997 0.2902489  0.28490722 0.26941108 0.23983273]
-- Palabras más similares: ['nasa', 'shuttle', 'seds', 'enfant', 'exploration']

Palabra: god
-- Similitud coseno con las 5 más similares: [0.28062614 0.27638988 0.266832   0.25928054 0.2588725 ]
-- Palabras más similares: ['jesus', 'bible', 'christ', 'faith', 'existence']



### Conclusión

Al analizar la similitud coseno entre palabras a partir de la matriz término-documento, se observa que las palabras elegidas tienden a agruparse con términos relacionados temáticamente. Por ejemplo, hockey aparece cercano a ncaa y nhl, game a games y team, space a nasa y shuttle, y god a jesus y bible. Esto indica que la vectorización TF-IDF logra capturar asociaciones semánticas entre palabras, reflejando la relación de términos dentro de los documentos del corpus. Algunas palabras, como computer, muestran vecinos menos intuitivos, lo que sugiere que para términos más raros o menos frecuentes, la similitud puede ser menos interpretativa.